# Problema del viajero
## Definición de la función fitness

In [195]:
import numpy as np

def fitness(distancias: np.ndarray):
    return np.sum(distancias, axis=1)

## Definición de cromosoma
Se compone de una lista que contiene el orden en que visita las ciudades:

- 0: Ciudad de México
- 1: Montreal
- 2: Moscú
- 3: Nueva York
- 4: París
- 5: Río de Janeiro
- 6: Roma

In [196]:
NUM_CIUDADES = 7
BASE = np.array(range(0, NUM_CIUDADES))
RNG = np.random.default_rng()
DISTANCIAS = np.array([
    [0, 2318, 6663, 2094, 5716, 4771, 6366],
    [2318, 0, 4386, 320, 3422, 5097, 4080],
    [6663, 4386, 0, 4065, 1544, 7175, 1474],
    [2094, 320, 4065, 0, 3624, 4817, 4281],
    [5716, 3422, 1544, 3624, 0, 5699, 697],
    [4771, 5097, 7175, 4817, 5699, 0, 5684],
    [6366, 4080, 1474, 4281, 697, 5684, 0]
])

class Cromosoma:
    """
    Se conforma por una lista que representa el orden en el que recorre las ciudades, cada número no debe repetirse
    """
    def __init__(self, ciudades=None):
        if ciudades is None:
            self.list = RNG.permutation(BASE)
        else:
            self.list = ciudades


    def __str__(self):
        return str(self.list)


    def get_distancias(self):
        distancias = [DISTANCIAS[self.list[i]][self.list[i+1]] for i in range(0, NUM_CIUDADES-1)]
        distancias.append(DISTANCIAS[self.list[-1]][self.list[0]])  # Para regresar al origen
        return np.array(distancias)


    @staticmethod
    def crossover_parcial(c1, c2):
        offspring_1 = -np.ones_like(c1.list)
        offspring_2 = -np.ones_like(c2.list)
        index = np.random.randint(0, high=NUM_CIUDADES-3)

        equivalencia_1 = c2.list[index:index+3].copy()
        equivalencia_2 = c1.list[index:index+3].copy()
        offspring_1[index:index+3] = equivalencia_1
        offspring_2[index:index+3] = equivalencia_2

        for i in range(0, NUM_CIUDADES):
            iterador = c1.list[i]

            if offspring_1[i] != -1:
                continue
            elif iterador in offspring_1:
                while iterador in offspring_1:
                    iterador = int(equivalencia_2[equivalencia_1 == iterador])

            offspring_1[i] = iterador

        for i in range(0, NUM_CIUDADES):
            iterador = c2.list[i]

            if offspring_2[i] != -1:
                continue
            elif iterador in offspring_2:
                while iterador in offspring_2:
                    iterador = int(equivalencia_1[equivalencia_2 == iterador])

            offspring_2[i] = iterador

        return [Cromosoma(offspring_1), Cromosoma(offspring_2)]

    @staticmethod
    def mutacion_desplazada(c):
        desde = np.random.randint(0, high=NUM_CIUDADES-3)
        hasta = np.random.randint(0, high=NUM_CIUDADES-3)
        temporal = c.list[hasta:hasta+3].copy()
        c.list[hasta:hasta+3] = c.list[desde:desde+3]
        c.list[desde:desde+3] = temporal

    @staticmethod
    def mutacion_intercambio(c):
        i = np.random.randint(low=0, high=NUM_CIUDADES)
        j = np.random.randint(low=0, high=NUM_CIUDADES)
        temporal = c.list[i]
        c.list[i] = c.list[j]
        c.list[j] = temporal


## Definición de presión selectiva

In [197]:
K_POBLACION = 8
K_BASE = 2
K_PROBABILIDAD_MUTACION = 0.5

def presion_selectiva(poblacion: list[Cromosoma]) -> list[Cromosoma]:
    # Evaluación y búsqueda del mejor
    distancias = np.array([c.get_distancias() for c in poblacion])
    evaluacion = fitness(distancias)

    best = evaluacion.argmin()
    print("Best so far:")
    print(f"Combination: {poblacion[best]}")
    print(f"Distances: {distancias[best]}")
    print(f"Evaluation: {evaluacion[best]}")

    # Cálculo de probabilidades
    indice_ordenado = evaluacion.argsort()
    ruleta = []
    potencia = K_POBLACION

    for i in indice_ordenado:
        probabilidad = K_BASE ** potencia
        ruleta.extend([i] * probabilidad)
        potencia -= 1

    # Nueva generación
    nueva = list[Cromosoma]()
    nueva.append(poblacion[indice_ordenado[0]])
    nueva.append(poblacion[indice_ordenado[1]])

    for i in range(1, int(K_POBLACION/2)):
        c1 = poblacion[np.random.choice(ruleta)]
        c2 = poblacion[np.random.choice(ruleta)]
        hijos = Cromosoma.crossover_parcial(c1, c2)

        for hijo in hijos:
            if np.random.choice([True, False], p=[K_PROBABILIDAD_MUTACION, 1-K_PROBABILIDAD_MUTACION]):
                Cromosoma.mutacion_intercambio(hijo)

        nueva.extend(hijos)

    return nueva

## Ciclo de vida

In [198]:
poblacion = list[Cromosoma]()
nueva_poblacion = list[Cromosoma]()
generacion = 0

In [215]:
if len(nueva_poblacion) == 0:
    poblacion = [Cromosoma() for _ in range(0, K_POBLACION)]
else:
    poblacion = nueva_poblacion

print('Generation', generacion)
nueva_poblacion = presion_selectiva(poblacion)
generacion += 1

Generation 16
Best so far:
Combination: [0 5 4 6 2 3 1]
Distances: [4771 5699  697 1474 4065  320 2318]
Evaluation: 19344
